In [18]:
%%configure -f
{
    "conf":{
        "spark.executor.instances": "4",
        "spark.executor.memory": "2g",
        "spark.executor.cores": "1"
    }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
556,application_1761923966900_0568,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
555,application_1761923966900_0567,pyspark,busy,Link,Link,None,
556,application_1761923966900_0568,pyspark,idle,Link,Link,None,✔


Query 2

Ανά έτος, να βρεθούν τα 3 φυλετικά γκρουπ με τα περισσότερα θύματα καταγεγραμμένων εγκλημάτων
(Vict Descent) στο Los Angeles. Τα αποτελέσματα να εμφανιστούν με φθίνουσα σειρά αριθμού θυμάτων
ανά φυλετικό γκρουπ – να υπολογιστεί και να εμφανιστεί επίσης το ποσοστό επί του συνολικού αριθμού θυμάτων ανα περίπτωση (δείτε παράδειγμα αποτελέσματος στον Πίνακα 2).

year Victim Descent # %

2024 White 413 32.5

2024 Black 274 25.4

2024 Unknown 132 22.3

2023 Hispanic/Latin/Mexican 512 30.2

In [34]:
#implementation with Dataframes
import time
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql import Row
#για μετρηση επιδοσης ολου του implementation
start_time = time.time()
spark = SparkSession \
    .builder \
    .appName("Query 1 implementation w Dataframes") \
    .getOrCreate()

# Define the schema for the employees DataFrame
crimes_schema = StructType([
    StructField("dr_no", StringType()),
    StructField("date_rptd", StringType()),
    StructField("date_occ", StringType()),
    StructField("time_occ", StringType()),
    StructField("area", StringType()),
    StructField("area_name", StringType()),
    StructField("rpt_dist_no", StringType()),
    StructField("part_1_2", IntegerType()),
    StructField("crm_cd", StringType()),
    StructField("crm_cd_desc", StringType()),
    StructField("mocodes", StringType()),
    StructField("vict_age", StringType()),
    StructField("vict_sex", StringType()),
    StructField("vict_descent", StringType()),
    StructField("premis_cd", StringType()),
    StructField("premis_desc", StringType()),
    StructField("weapon_used_cd", StringType()),
    StructField("weapon_desc", StringType()),
    StructField("status", StringType()),
    StructField("status_desc", StringType()),
    StructField("crm_cd_1",StringType()),
    StructField("crm_cd_2",StringType()),
    StructField("crm_cd_3",StringType()),
    StructField("crm_cd_4",StringType()),
    StructField("location", StringType()),
    StructField("cross_street", StringType()),
    StructField("lat", FloatType()),
    StructField("lon", FloatType()),
])

#Δημιουργούμε ενα λεξικό που αντιστοιχίζει τον κωδικό μεε το φυλετικό γκρούπ
descent_dict = {
    "A": "Other Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian"
}
#Μετατρεπουμε το λεξικό σε Dataframe και μετα θα το κανουμε left join για να αντιστοιχίσουμε τον κωδικο με το γκρουπ
descent_rows = [(k, v) for k, v in descent_dict.items()]
descent_df = spark.createDataFrame(descent_rows, ["vict_descent", "descent_description"])

crimes_2010_2019_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv", \
    header=False, \
    schema=crimes_schema)

crimes_2020_2025_df = spark.read.csv("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv", \
                                     header=False, \
                                     schema=crimes_schema)

#Union both datasets to have all the data available for the query
crimes_total_df = crimes_2010_2019_df.union(crimes_2020_2025_df)

#Απο το StringType προσθέτουμε καινουριο column "year" που ουσιαστικά ειναι οι πρωτοι 4 χαρακτήρες του sting απο date_occ
years_df = crimes_total_df.withColumn("year", F.substring("date_occ", 1, 4).cast("int"))

#years_df.show(10)
#Για καθε χρονια θα βρουμε τις τρεις φυλετικες κατηγοριες με τα περισσοτερα περιστατικά
#θα βαλουμε ολες τις χρονιες σε μια λιστα και μετα για καθε στοιχειο τις λιστας που συμπηπτει με τον χρονο στο 

def top3_racial_groups_per_year(df):

    
    # 2. get distinct years
    years = [row["year"] for row in df.select("year").distinct().collect()]
    
    results = []
    
    for y in years:
        # όλα τα incidents του έτους y
        df_year = df.filter(F.col("year") == y)
        
        total = df_year.count()
        if total == 0:
            continue
        
        # count per racial group
        groups = (
            df_year.groupBy("vict_descent")
                   .count()
                   .orderBy(F.col("count").desc())
        )
        
        top3 = groups.limit(3).collect()
        
        # αποθήκευση αποτελεσμάτων
        for row in top3:
            vict = row["vict_descent"]
            c = row["count"]
            pct = (c / total) * 100
            
            results.append(
                Row(
                    year=y,
                    vict_descent=vict,
                    count=c,
                    total=total,
                    percentage=pct
                )
            )
    
    # δημιουργεί τελικό DataFrame
    spark = df.sparkSession
    result_df = spark.createDataFrame(results)

    # ταξινόμηση αποτελεσμάτων
    result_df = result_df.join(descent_df, on="vict_descent", how="left")
    result_df = result_df.withColumn("descent_description", F.when(F.col("descent_description").isNull(), "Unknown (was not filed)").otherwise(F.col("descent_description")))
    result_df = result_df.drop("total").select("year","descent_description", "count", "percentage")
    result_df = result_df.orderBy(F.col("year").desc(), F.col("count").desc())
    return result_df

total_res_df = top3_racial_groups_per_year(years_df)
#result_df = result_df.drop("total").select("year","descent_description", "count", "percentage")
total_res_df.show()
end_time= time.time()
print("RDD API Execution time for Query 2: {:.4f} sec".format(end_time - start_time))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+--------------------+-----+------------------+
|year| descent_description|count|        percentage|
+----+--------------------+-----+------------------+
|2025|Hispanic/Latin/Me...|   34|35.051546391752574|
|2025|             Unknown|   24|24.742268041237114|
|2025|Unknown (was not ...|   13|13.402061855670103|
|2024|Unknown (was not ...|29204|22.893067956446416|
|2024|Hispanic/Latin/Me...|28576|22.400777630578443|
|2024|               White|22958|17.996817358721298|
|2023|Hispanic/Latin/Me...|69401| 29.86980567690288|
|2023|               White|44615|19.202048677613032|
|2023|Unknown (was not ...|31497|13.556134196991543|
|2022|Hispanic/Latin/Me...|73111|31.076813214372244|
|2022|               White|46695| 19.84833736435163|
|2022|               Black|34634|14.721647205845473|
|2021|Hispanic/Latin/Me...|63676| 30.33981970306276|
|2021|               White|44523|21.213954906706817|
|2021|               Black|30173|14.376584268806342|
|2020|Hispanic/Latin/Me...|61606|30.8265823354

In [35]:
# Implementation 2: SQL API
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, FloatType, StringType
import time 

start_time = time.time()
spark = SparkSession \
    .builder \
    .appName("Query 2 w SQL API") \
    .getOrCreate()

crimes_schema = StructType([
    StructField("dr_no", StringType()),
    StructField("date_rptd", StringType()),
    StructField("date_occ", StringType()),
    StructField("time_occ", StringType()),
    StructField("area", StringType()),
    StructField("area_name", StringType()),
    StructField("rpt_dist_no", StringType()),
    StructField("part_1_2", IntegerType()),
    StructField("crm_cd", StringType()),
    StructField("crm_cd_desc", StringType()),
    StructField("mocodes", StringType()),
    StructField("vict_age", StringType()),
    StructField("vict_sex", StringType()),
    StructField("vict_descent", StringType()),
    StructField("premis_cd", StringType()),
    StructField("premis_desc", StringType()),
    StructField("weapon_used_cd", StringType()),
    StructField("weapon_desc", StringType()),
    StructField("status", StringType()),
    StructField("status_desc", StringType()),
    StructField("crm_cd_1",StringType()),
    StructField("crm_cd_2",StringType()),
    StructField("crm_cd_3",StringType()),
    StructField("crm_cd_4",StringType()),
    StructField("location", StringType()),
    StructField("cross_street", StringType()),
    StructField("lat", FloatType()),
    StructField("lon", FloatType()),
])

crimes_2010_2019_df = spark.read.format('csv') \
    .options(header='false') \
    .schema(crimes_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2010_2019.csv")

crimes_2020_2025_df = spark.read.format('csv') \
    .options(header='false') \
    .schema(crimes_schema) \
    .load("s3://initial-notebook-data-bucket-dblab-905418150721/project_data/LA_Crime_Data/LA_Crime_Data_2020_2025.csv")

#Δημιουργούμε ενα λεξικό που αντιστοιχίζει τον κωδικό μεε το φυλετικό γκρούπ
descent_dict = {
    "A": "Other Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian"
}

#Μετατρεπουμε το λεξικό σε Dataframe και μετα θα το κανουμε left join για να αντιστοιχίσουμε τον κωδικο με το γκρουπ
descent_rows = [(k, v) for k, v in descent_dict.items()]
descent_df = spark.createDataFrame(descent_rows, ["vict_descent", "descent_description"])

crimes_total_df = crimes_2010_2019_df.union(crimes_2020_2025_df)

#Απο το StringType προσθέτουμε καινουριο column "year" που ουσιαστικά ειναι οι πρωτοι 4 χαρακτήρες του sting απο date_occ
years_df = crimes_total_df.withColumn("year", F.substring("date_occ", 1, 4).cast("int"))
# To utilize as SQL tables
years_df.createOrReplaceTempView("crimes")
descent_df.createOrReplaceTempView("descent")

#κανουμε join για να εχουμε το description
join_query = " \
    SELECT \
        c.year, \
        c.vict_descent, \
        COALESCE(d.descent_description, 'Unknown (was not filed)') AS descent_description \
    FROM crimes c \
    LEFT JOIN descent d \
        ON c.vict_descent = d.vict_descent \
"

joined_df = spark.sql(join_query)
joined_df.createOrReplaceTempView("joined")

count_query = " \
    SELECT \
        year, \
        vict_descent, \
        descent_description, \
        COUNT(*) AS count \
    FROM joined \
    GROUP BY year, vict_descent, descent_description \
"

counts_df = spark.sql(count_query)
counts_df.createOrReplaceTempView("counts")

totals_query = " \
    SELECT \
        *, \
        SUM(count) OVER (PARTITION BY year) AS total_per_year \
    FROM counts \
"

totals_df = spark.sql(totals_query)
totals_df.createOrReplaceTempView("with_totals")

rank_query = " \
    SELECT \
        year, \
        vict_descent, \
        descent_description, \
        count, \
        total_per_year AS total, \
        ROUND((count / total_per_year) * 100, 2) AS percentage, \
        ROW_NUMBER() OVER (PARTITION BY year ORDER BY count DESC) AS rn \
    FROM with_totals \
"

ranked_df = spark.sql(rank_query)
ranked_df.createOrReplaceTempView("ranked")

final_query = " \
    SELECT \
        year, \
        vict_descent, \
        descent_description, \
        count, \
        total, \
        percentage \
    FROM ranked \
    WHERE rn <= 3 \
    ORDER BY year DESC, count DESC \
"

result_sql_df = spark.sql(final_query)
result_sql_df.show(100, truncate=False)
result_sql_df.explain(mode='formatted')
end_time=time.time()
print("SQL API Execution time for Query 2: {:.4f} sec".format(end_time - start_time))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------------+-----------------------+-----+------+----------+
|year|vict_descent|descent_description    |count|total |percentage|
+----+------------+-----------------------+-----+------+----------+
|2025|H           |Hispanic/Latin/Mexican |34   |97    |35.05     |
|2025|X           |Unknown                |24   |97    |24.74     |
|2025|NULL        |Unknown (was not filed)|13   |97    |13.4      |
|2024|NULL        |Unknown (was not filed)|29204|127567|22.89     |
|2024|H           |Hispanic/Latin/Mexican |28576|127567|22.4      |
|2024|W           |White                  |22958|127567|18.0      |
|2023|H           |Hispanic/Latin/Mexican |69401|232345|29.87     |
|2023|W           |White                  |44615|232345|19.2      |
|2023|NULL        |Unknown (was not filed)|31497|232345|13.56     |
|2022|H           |Hispanic/Latin/Mexican |73111|235259|31.08     |
|2022|W           |White                  |46695|235259|19.85     |
|2022|B           |Black                  |34634